# 50. Create Plot and Documents RDF Resources
* Associate each table page with a cover page to retrieve section identifier
* Create the Landmark of type Plot
* Create the Record Resources of each line and page

In [1]:
from rdflib import Graph, URIRef, Literal
from pyvis.network import Network

In [2]:
import sys
import os
# Access to the utils directory
current_dir = os.getcwd()
utils_dir = os.path.join(current_dir, '..', 'utils')
sys.path.append(utils_dir)

In [3]:
COMMUNE = "LHAY"
DEP = "94"
DATA = f"/home/STual/DAN-cadastre/data/{COMMUNE}"

In [4]:
if not os.path.exists(f'{DATA}/viz'):
    os.makedirs(f'{DATA}/viz')

## 1. Landmarks (Section, Commune)

In [8]:
from rdflib import Graph, URIRef, Literal
from pyvis.network import Network

# Load your Turtle file
g = Graph()
g.parse(f"{DATA}/rdf/landmarks.ttl", format="turtle")  # Replace with your actual file

# Build a label map
label_map = {}

for s, p, o in g:
    # Check for labels
    if p.endswith("label") or p.endswith("prefLabel"):
        label_map[str(s)] = str(o)

# Fallback function to make a label from a URI
def short_label(uri):
    return label_map.get(uri, uri.split("/")[-1].split("#")[-1])

# Initialize Pyvis graph
net = Network(notebook=True, directed=True)

# Add nodes and edges
for s, p, o in g:
    s_label = short_label(str(s))
    p_label = short_label(str(p))
    o_label = short_label(str(o))

    net.add_node(s_label, label=s_label)
    net.add_node(o_label, label=o_label)
    net.add_edge(s_label, o_label, label=p_label)

# Show graph
net.show(f"landmarks.html")

landmarks.html


## 2. Sources
* Look at the subgraph of a given page Instanciation

In [22]:
def get_type_label(uri):
    for o in g.objects(uri, RDF.type):
        # Use qname for better readability: rico:Instantiation, etc.
        return g.qname(o)
    return "other"

In [35]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF
from pyvis.network import Network

KEEP_PREDICATES = {
    "rico:identifier",
    "rico:isOrWasIncludedIn",
    "rico:isOrWasDigitalInstantiationOf",
    "rico:hasCreationDate",
}

# Load RDF file
g = Graph()
g.parse(f"{DATA}/rdf/sources.ttl", format="turtle")

# Define namespaces
RICO = Namespace("https://www.ica.org/standards/RiC/ontology#")
SOURCE = Namespace("http://rdf.geohistoricaldata.org/id/source/")

# Select instantiation with specific identifier
target_id = "3P275"
instantiations = []

for s in g.subjects(RICO.identifier, Literal(target_id)):
    instantiations.append(s)

In [36]:
if instantiations:
    inst = instantiations[0]
    connected_triples = set()

    for p, o in g.predicate_objects(inst):
        connected_triples.add((inst, p, o))
        
        # Get one hop further for related nodes
        if isinstance(o, URIRef):
            for pp, oo in g.predicate_objects(o):
                connected_triples.add((o, pp, oo))
else:
    print("No matching instantiation found.")

In [37]:
label_map = {}

# Collect human-readable labels
for s, p, o in g:
    if p.endswith("label") or p.endswith("name") or p.endswith("identifier"):
        label_map[str(s)] = str(o)

def get_label(uri):
    return label_map.get(str(uri), str(uri).split("/")[-1].split("#")[-1])

In [40]:
net = Network(notebook=True, directed=True)
nodes = set()

for s, p, o in connected_triples:
    s_label = get_label(s)
    p_label = get_label(p)
    o_label = get_label(o)

    # Add nodes if not already added
    if s_label not in nodes:
        net.add_node(s_label, label=s_label)
        nodes.add(s_label)

    if o_label not in nodes:
        net.add_node(o_label, label=o_label)
        nodes.add(o_label)

    net.add_edge(s_label, o_label, label=p_label)

In [41]:
net.show("sources.html")

sources.html
